## Úkol 3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shippy/czechitas-ai-data/blob/main/notebooks/assignment-03.ipynb)


### Nastavte si API klíč

1. Navštivte [Google AI Studio](https://aistudio.google.com/app/apikey) a vytvořte si API klíč.
2. V levé liště klikněte na tlačítko "Secrets" a přidejte předtím vytvořený API klíč do proměnné `GOOGLE_API_KEY` - buď "napřímo" přes "+ Add new secret", nebo přes "Gemini API keys" > "Import key from Google AI Studio".

(Pokud jste otevřeli tento notebook lokálně, např. ve VSCode, můžete si API klíč nastavit v souboru `.env`.)

### Setup a test

Spusťte následující dvě buňky. Pokud vám AI řekne, jak se má, vše funguje správně.

In [2]:
try:
    from google.colab import userdata
    _secret = userdata.get("GOOGLE_API_KEY")
    %pip install instructor google-generativeai python-dotenv rich
except ImportError:
    import os
    from dotenv import load_dotenv
    _ = load_dotenv()
    _secret = os.environ.get("GOOGLE_API_KEY")

In [3]:
import instructor
import google.generativeai as genai
import os
from rich import print

try:
    if _secret:
        genai.configure(api_key=_secret)
    else:
        raise ValueError("API klíč nebyl nastaven!")
except NameError:
    print("Nastavte si API klíč v proměnné GOOGLE_API_KEY a znovu spusťte *celý* notebook včetně předchozí buňky")

_client = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash-latest",
)
# _client.generate_content pro "normální" použití
client = instructor.from_gemini(
    client=_client,
    mode=instructor.Mode.GEMINI_JSON,
)
test = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Hello, how are you? Respond with an emotion."},
    ],
    response_model=str,
)
print(test)


/Users/simon/Documents/czechitas-ai-data/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello! I am doing well, thank you for asking. 😊

### Úkol, část 1

Vytáhněte ze nestrukturovaných odpovědí níže definované datové struktury.

In [4]:
from pydantic import BaseModel, Field
from typing import Literal

Occupation = Literal["gardener", "teacher", "doctor", "engineer", "lawyer", "scientist", "artist", "writer", "pensioner", "other"]


class Citizen(BaseModel):
    name: str = Field(..., description="Full name of the citizen")
    age: int
    occupation: Occupation
    interests: list[str]

In [5]:
census_responses = [
    "Jsem osmadvacetiletý Honza Pálka, pracuju v Semilhotce jako čistič kanálů. Rád hraju šachy a na bicí.",
    "Jsem Jana Nováková, je mi 30, dojíždím do Semilhotské školy učit. Ráda čtu knihy a jezdím na hory.",
    "Jsem čtyřicetiletý Petr Válek, pracuju v Semilhotce jako zedník. Rád cestuji a hraju fotbal.",
    "Jsem padesátiletý Vladimír Dvořák, pracuju v Semilhotce jako policista. Rád cestuji a hraju kulečník.",
    "Jsem šedesátiletý Jaroslav Bílek, pracuju v Semilhotce jako zahradník. Rád cestuji a hraju kulečník.",
    "Jsem sedmdesátiletý Karel Dvořák, pracuju v Semilhotce jako zedník. Rád cestuji a hraju kulečník.",
    "Jsem osmdesátiletý Josef Novák, pracuju v Semilhotce jako inženýr. Rád cestuji a hraju kulečník.",
    "Jsem devadesátiletý Igor Hnízdo, jsem tu penzista. Rád cestuji a hraju kulečník.",
]


- Tip 1: můžete použít i prompt engineering (tj. první message s `"role": "system"`), aby AI lépe poznala, co chcete.
- Tip 2: `client.chat.completions.create` umí vracet i `response_model=Citizen` nebo i `response_model=list[Citizen]`

Dávalo by smysl na základě dat přidefinovat do třídy `Citizen` nějaký atribut, abyste zachytili více informací?

In [ ]:
# Zde řešte úkol 1.
citizens = []

for response in census_responses:
    result = client.chat.completions.create(
        messages=[
            {"role": "user", "content": response},
        ],
        response_model=Citizen,
    )
    citizens.append(result)

print(citizens)


[
    Citizen(name='Honza Pálka', age=28, occupation='other', interests=['šachy', 'bicí']),
    Citizen(
        name='Jana Nováková',
        age=30,
        occupation='teacher',
        interests=['reading books', 'hiking in the mountains']
    ),
    Citizen(name='Petr Válek', age=40, occupation='other', interests=['cestovani', 'fotbal']),
    Citizen(name='Vladimír Dvořák', age=50, occupation='other', interests=['cestování', 'kulečník']),
    Citizen(name='Jaroslav Bílek', age=60, occupation='gardener', interests=['traveling', 'playing billiards']),
    Citizen(name='Karel Dvořák', age=70, occupation='other', interests=['cestování', 'kulečník']),
    Citizen(name='Josef Novák', age=80, occupation='engineer', interests=['cestovani', 'kulečník']),
    Citizen(name='Igor Hnízdo', age=90, occupation='pensioner', interests=['cestování', 'kulečník'])
]

### Úkol, část 2

Vygenerujte dataset nebo sadu datasetů, který popíše fiktivní městečko Semilhotka. Začněte např. s datovou strukturou `Citizen` následovně:

In [7]:

# A vygenerujeme nějaké obyvatele
citizens = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Generate 10 citizens of Semilhotka."},
    ],
    response_model=list[Citizen],
)
print(citizens)

[
    Citizen(name='Vasyl Romanov', age=62, occupation='pensioner', interests=['fishing', 'gardening', 'chess']),
    Citizen(name='Iryna Petrenko', age=35, occupation='teacher', interests=['reading', 'hiking', 'yoga']),
    Citizen(
        name='Petro Shevchenko',
        age=48,
        occupation='engineer',
        interests=['programming', 'cycling', 'photography']
    ),
    Citizen(name='Olena Kovalenko', age=28, occupation='doctor', interests=['medicine', 'traveling', 'dancing']),
    Citizen(name='Andriy Marchenko', age=55, occupation='lawyer', interests=['law', 'history', 'politics']),
    Citizen(name='Tetiana Hrytsenko', age=31, occupation='scientist', interests=['research', 'writing', 'music']),
    Citizen(name='Oleh Melnyk', age=42, occupation='artist', interests=['painting', 'sculpture', 'drawing']),
    Citizen(name='Hanna Zaitseva', age=24, occupation='writer', interests=['literature', 'poetry', 'theater']),
    Citizen(
        name='Serhiy Kuzmenko',
        age=68,
        occupation='gardener',
        interests=['gardening', 'birdwatching', 'fishing']
    ),
    Citizen(
        name='Nataliia Levchenko',
        age=45,
        occupation='other',
        interests=['sewing', 'cooking', 'volunteering']
    )
]

Ideálně bychom chtěli, aby výčet kulminoval objektem Semilhotka:

```python
from pydantic import BaseModel

class Semilhotka(BaseModel):
    citizens: list[Citizen]
    # a další atributy

my_semilhotka = Semilhotka(
    citizens=citizens,
    # a další proměnné, které jste si vygenerovali
)
```

In [8]:
# Zde řešte úkol 2. (Můžete si vytvořit i více buněk!)
class Building(BaseModel):
    name: str
    type: Literal["house", "school", "hospital", "police_station", "church", "town_hall", "park", "cemetery"]
    address: str

buildings = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Generate 5 buildings in Semilhotka."},
    ],
    response_model=list[Building],
)


In [9]:

class Semilhotka(BaseModel):
    citizens: list[Citizen]
    mayor: Citizen
    buildings: list[Building]

my_semilhotka = Semilhotka(
    citizens=citizens,
    mayor=citizens[0],
    buildings=buildings,
)
print(my_semilhotka)

Semilhotka(
    citizens=[
        Citizen(
            name='Vasyl Romanov',
            age=62,
            occupation='pensioner',
            interests=['fishing', 'gardening', 'chess']
        ),
        Citizen(name='Iryna Petrenko', age=35, occupation='teacher', interests=['reading', 'hiking', 'yoga']),
        Citizen(
            name='Petro Shevchenko',
            age=48,
            occupation='engineer',
            interests=['programming', 'cycling', 'photography']
        ),
        Citizen(
            name='Olena Kovalenko',
            age=28,
            occupation='doctor',
            interests=['medicine', 'traveling', 'dancing']
        ),
        Citizen(name='Andriy Marchenko', age=55, occupation='lawyer', interests=['law', 'history', 'politics']),
        Citizen(
            name='Tetiana Hrytsenko',
            age=31,
            occupation='scientist',
            interests=['research', 'writing', 'music']
        ),
        Citizen(name='Oleh Melnyk', age=42, occupation='artist', interests=['painting', 'sculpture', 'drawing']),
        Citizen(
            name='Hanna Zaitseva',
            age=24,
            occupation='writer',
            interests=['literature', 'poetry', 'theater']
        ),
        Citizen(
            name='Serhiy Kuzmenko',
            age=68,
            occupation='gardener',
            interests=['gardening', 'birdwatching', 'fishing']
        ),
        Citizen(
            name='Nataliia Levchenko',
            age=45,
            occupation='other',
            interests=['sewing', 'cooking', 'volunteering']
        )
    ],
    mayor=Citizen(
        name='Vasyl Romanov',
        age=62,
        occupation='pensioner',
        interests=['fishing', 'gardening', 'chess']
    ),
    buildings=[
        Building(name='Semilhotka Primary School', type='school', address='123 School St, Semilhotka'),
        Building(name="St. Michael's Church", type='church', address='456 Church Ln, Semilhotka'),
        Building(name='Semilhotka Town Hall', type='town_hall', address='789 Main St, Semilhotka'),
        Building(name='Semilhotka General Hospital', type='hospital', address='101 Hospital Rd, Semilhotka'),
        Building(name='Oakhaven Cemetery', type='cemetery', address='222 Oak Ave, Semilhotka')
    ]
)

## Export dat

Pro jednoduchost si vaši vesnici exportujeme jako Python pickle soubor. Ten si potom z levého panelu můžete stáhnout a použít v jiném notebooku (jste-li  v Colabu).

In [10]:
from pickle import dump

try:
    if my_semilhotka:
        with open("semilhotka.pkl", "wb") as f:
            dump(my_semilhotka, f)
except NameError:
    raise ValueError("Musíte si nejdříve nadefinovat Semilhotku a přiřadit ji do proménně!")
